## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-04-04-06-26 +0000,nyt,Texas Democrats Leave State to Stop Republican...,https://www.nytimes.com/2025/08/03/us/texas-de...
1,2025-08-04-04-06-00 +0000,wsj,Democrats in the Texas state House fled the st...,https://www.wsj.com/politics/policy/texas-demo...
2,2025-08-04-04-00-10 +0000,nyt,He Survived the Khmer Rouge and Built a Musica...,https://www.nytimes.com/2025/08/04/arts/music/...
3,2025-08-04-04-00-05 +0000,nyt,What’s It Like to Deal With Brutal U.S. Tariff...,https://www.nytimes.com/2025/08/04/business/ma...
4,2025-08-04-04-00-05 +0000,nypost,Trump praises Sydney Sweeney after learning sh...,https://nypost.com/2025/08/04/us-news/trump-pr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2343/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,31
6,house,8
59,new,8
63,st,7
438,attack,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
132,2025-08-03-14-22-11 +0000,nypost,Trump rips ‘sleazebag’ Charlamagne tha God aft...,https://nypost.com/2025/08/03/us-news/trump-ri...,80
23,2025-08-04-00-50-00 +0000,wsj,A chief White House economic adviser said that...,https://www.wsj.com/economy/jobs/trump-seeks-b...,75
1,2025-08-04-04-06-00 +0000,wsj,Democrats in the Texas state House fled the st...,https://www.wsj.com/politics/policy/texas-demo...,72
15,2025-08-04-01-57-10 +0000,nypost,Diehard soccer fan kicked out of St. Louis sta...,https://nypost.com/2025/08/03/us-news/diehard-...,59
14,2025-08-04-02-03-23 +0000,nyt,Trump ‘wants his own people’ in the Bureau of ...,https://www.nytimes.com/live/2025/08/03/us/tru...,57


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
132,80,2025-08-03-14-22-11 +0000,nypost,Trump rips ‘sleazebag’ Charlamagne tha God aft...,https://nypost.com/2025/08/03/us-news/trump-ri...
23,44,2025-08-04-00-50-00 +0000,wsj,A chief White House economic adviser said that...,https://www.wsj.com/economy/jobs/trump-seeks-b...
148,43,2025-08-03-10-34-36 +0000,nypost,Ukrainian drone attack sparks massive fire at ...,https://nypost.com/2025/08/03/world-news/ukrai...
172,40,2025-08-03-07-39-06 +0000,nypost,US envoy tells Israeli hostage families he’s w...,https://nypost.com/2025/08/03/world-news/us-en...
91,33,2025-08-03-18-17-00 +0000,cbc,Krasheninnikov volcano in Russia's Far East er...,https://www.cbc.ca/news/world/kamchatka-penins...
160,33,2025-08-03-09-30-00 +0000,wsj,"Like Zohran Mamdani in New York, the rise of M...",https://www.wsj.com/politics/elections/sociali...
1,28,2025-08-04-04-06-00 +0000,wsj,Democrats in the Texas state House fled the st...,https://www.wsj.com/politics/policy/texas-demo...
70,27,2025-08-03-20-37-30 +0000,bbc,Manhunt intensifies for Montana bar shooting s...,https://www.bbc.com/news/articles/cy8j79yw8pqo...
62,26,2025-08-03-21-13-37 +0000,nyt,Third Child in Sailing Camp Dies After Barge H...,https://www.nytimes.com/2025/08/03/us/miami-be...
178,26,2025-08-03-04-27-43 +0000,bbc,"Their children can't eat, speak or walk - so f...",https://www.bbc.com/news/articles/c39d0zj0110o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
